<a href="https://colab.research.google.com/github/inoueshinichi/PytorchTutorial/blob/main/PytorchTutorial_p1ch2_pretrainednetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pythonのバージョンチェック

In [3]:
!python

Python 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
^C


In [4]:
print('Hello, world!')

Hello, world!


In [5]:
print("Until next time...")

Until next time...


インポート設定

In [19]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

第二章  
・画像認識の訓練済みモデルの実行方法  
・GANとCycleGANについて解説  
・画像に写っている内容を説明する、テキスト生成モデルの解説  
・Torch Hubでのモデル共有方法

In [20]:
from torchvision import models

In [21]:
dir(models) # 用意されている訓練済みモデルの一覧を見る

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'MobileNetV3',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'mobilenetv2',
 'mobilenetv3',
 'quantization',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 '

AlexNet

In [22]:
alexnet = models.AlexNet() # AlexNetクラスのインスタンスを作成

ResNet

In [23]:
resnet = models.resnet101(pretrained=True) # 120万枚の画像と1000個のカテゴリからなるImageNetデータセットで学習された101層のResNetクラスのインスタンスを作成

In [24]:
print(resnet) # モデル構造を確認

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

入力データの前処理

In [25]:
from torchvision import transforms
preprocess = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
  )
])

クラウドサーバーにアップロードしたRGB画像ファイルを確認

In [26]:
!pwd

/content


In [27]:
!ls -la /content/upload_data/image

total 2360
drwxr-xr-x 2 root root   4096 Jul  5 15:32 .
drwxr-xr-x 4 root root   4096 Jul  5 15:34 ..
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Aerial.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Airplane.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Balloon.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 couple.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Earth.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Girl.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Lenna.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Mandrill.bmp
-rw-r--r-- 1 root root 196664 Jul  5 15:32 milkdrop.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Parrots.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Pepper.bmp
-rw-r--r-- 1 root root 196662 Jul  5 15:32 Sailboat.bmp


アップロードしたRGB画像を読み込む

In [28]:
from PIL import Image
img = Image.open("./upload_data/image/Aerial.bmp")
print("Aerial.bmp\n", img)

Aerial.bmp
 <PIL.BmpImagePlugin.BmpImageFile image mode=RGB size=256x256 at 0x7F7B301DEE50>


In [29]:
img.show() # 動かない

画像を前処理パイプラインに通す

In [30]:
img_t = preprocess(img)
print(img_t)

tensor([[[ 0.5022,  0.6049,  ...,  0.0569,  0.0912],
         [ 0.1083,  0.0912,  ..., -0.1314,  0.2624],
         ...,
         [-0.9534, -0.8849,  ...,  0.0912,  0.5878],
         [-0.8507, -0.9192,  ...,  0.5022,  0.3994]],

        [[ 0.7304,  1.0455,  ...,  0.8179,  0.7304],
         [ 0.4853,  0.6954,  ...,  0.7654,  0.9055],
         ...,
         [-0.7577, -0.8277,  ...,  1.1506,  1.4657],
         [-0.7752, -0.8102,  ...,  1.3957,  1.2031]],

        [[ 0.6356,  0.8448,  ...,  1.0191,  0.9842],
         [ 0.6182,  0.8622,  ...,  1.0888,  1.1585],
         ...,
         [ 0.0431,  0.0256,  ...,  1.5420,  1.7511],
         [ 0.0605, -0.0964,  ...,  1.6117,  1.3851]]])


img_tをunsqueezeする

In [32]:
import torch
batch_t = torch.unsqueeze(img_t, 0)
print(batch_t)
print(batch_t.size())

tensor([[[[ 0.5022,  0.6049,  ...,  0.0569,  0.0912],
          [ 0.1083,  0.0912,  ..., -0.1314,  0.2624],
          ...,
          [-0.9534, -0.8849,  ...,  0.0912,  0.5878],
          [-0.8507, -0.9192,  ...,  0.5022,  0.3994]],

         [[ 0.7304,  1.0455,  ...,  0.8179,  0.7304],
          [ 0.4853,  0.6954,  ...,  0.7654,  0.9055],
          ...,
          [-0.7577, -0.8277,  ...,  1.1506,  1.4657],
          [-0.7752, -0.8102,  ...,  1.3957,  1.2031]],

         [[ 0.6356,  0.8448,  ...,  1.0191,  0.9842],
          [ 0.6182,  0.8622,  ...,  1.0888,  1.1585],
          ...,
          [ 0.0431,  0.0256,  ...,  1.5420,  1.7511],
          [ 0.0605, -0.0964,  ...,  1.6117,  1.3851]]]])
torch.Size([1, 3, 224, 224])


試しに訓練済みResNet101にAerial.bmp画像を入力して推論結果を取得する

In [37]:
resnet.eval() # 推論を行うためにネットワークを`eval`モードに切り替える

# ネットワークをevalモードにするのを忘れてしまうと、バッチ正規化やドロップアウトを含む
# 訓練済みモデルでは、それらのモジュール性質上、意味のある出力が得られない場合がある。

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [39]:
# 推論
out = resnet(batch_t)
out # ネットワークの4,450万個のパラメータを含む計算が行われ、1000個のカテゴリ毎の確率が得られる

tensor([[-2.0035e+00,  5.2557e+00,  2.1686e+00,  3.7032e+00,  4.6461e+00,
          5.9203e+00,  6.8438e+00,  7.0077e-01,  4.1159e-01, -2.0327e+00,
         -2.4331e+00, -2.1634e+00,  6.4096e-01, -6.2453e-01, -1.2262e+00,
          3.0978e-01, -2.9524e+00, -1.1965e+00,  2.5533e-01, -1.0956e+00,
         -2.3133e+00, -2.2174e+00, -1.3565e+00, -7.1513e-01, -1.0508e+00,
         -7.0858e-01, -7.5757e-01,  1.4600e+00, -6.8343e-01, -3.4401e-01,
         -1.5859e+00, -1.6408e+00, -7.1287e-01,  8.1602e+00,  4.3667e+00,
          1.4516e+00,  7.2398e-01,  1.1716e+00,  1.0074e+00,  2.1030e+00,
          2.1941e-01, -2.1451e-01,  1.1805e+00,  6.0272e-02,  2.8499e-01,
          1.8677e+00,  9.9590e-01, -1.0459e+00, -1.3204e+00, -3.0580e+00,
         -7.2645e-01, -1.9732e+00,  1.1656e+00, -2.1199e-02,  1.5701e+00,
         -6.1431e-02, -1.0486e+00,  1.3616e-01,  2.8926e+00, -1.4453e+00,
          1.8180e+00, -8.1043e-02,  2.2455e-01, -1.2266e+00, -1.7295e-01,
          7.4045e+00, -1.3791e+00,  5.

予測されたラベルのリストを確認するために、  
訓練中にネットワークに提示したものと同じ順番で  
ラベルをリストアップしたテキストファイルをロードする.

In [40]:
with open("./upload_data/imagenet_classes.txt") as f:
  labels = [line.strip() for line in f.readlines()]
labels

['tench, Tinca tinca',
 'goldfish, Carassius auratus',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'tiger shark, Galeocerdo cuvieri',
 'hammerhead, hammerhead shark',
 'electric ray, crampfish, numbfish, torpedo',
 'stingray',
 'cock',
 'hen',
 'ostrich, Struthio camelus',
 'brambling, Fringilla montifringilla',
 'goldfinch, Carduelis carduelis',
 'house finch, linnet, Carpodacus mexicanus',
 'junco, snowbird',
 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 'robin, American robin, Turdus migratorius',
 'bulbul',
 'jay',
 'magpie',
 'chickadee',
 'water ouzel, dipper',
 'kite',
 'bald eagle, American eagle, Haliaeetus leucocephalus',
 'vulture',
 'great grey owl, great gray owl, Strix nebulosa',
 'European fire salamander, Salamandra salamandra',
 'common newt, Triturus vulgaris',
 'eft',
 'spotted salamander, Ambystoma maculatum',
 'axolotl, mud puppy, Ambystoma mexicanum',
 'bullfrog, Rana catesbeiana',
 'tree frog, tree-f

出力テンソルの最大スコアに対応するインデックスを求める

In [41]:
max_val, max_idx = torch.max(out, 1)
print(f"max_val: {max_val}")
print(f"max_idx: {max_idx}")

max_val: tensor([10.1488], grad_fn=<MaxBackward0>)
max_idx: tensor([973])


In [42]:
# スコアを[0,1]に正規化して、その合計で割ってパーセンテージを取得する
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100 # (1000,)
print(f"inf category: {labels[max_idx[0]]}, inf %: {percentage[max_idx[0]].item()}")

inf category: coral reef, inf %: 36.67506408691406


推論結果の2番手, 3番手の候補も見てみる

In [43]:
_, indices = torch.sort(out, descending=True) # 降順
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('coral reef', 36.67506408691406),
 ('lionfish', 14.735217094421387),
 ('wreck', 8.995096206665039),
 ('scuba diver', 6.157462120056152),
 ('loggerhead, loggerhead turtle, Caretta caretta', 5.020261287689209)]